In [ ]:
import numpy as np
from pathlib import Path
from tqdm.auto import tqdm
from fastai.data.all import get_image_files
from fastai.vision.all import load_learner,Image, PILImage

In [ ]:
PROJ_PATH = Path.cwd().parent
DATASET_NAME = 'MAGNETIC_TILE_SURFACE_DEFECTS'
DATASET_PATH = PROJ_PATH/'data'/DATASET_NAME

In [ ]:
def get_mask_path(img_path): 
    msk_path = img_path.parent.parent/'masks'/f'{img_path.stem}.png'
    return msk_path

In [ ]:
pickle_path = DATASET_PATH/'export_learn_obj.pkl'
learn = load_learner(pickle_path, cpu=True)

In [ ]:
def dice_coef(y_true, y_pred, eps=0.02):
    intersection = np.sum(y_true * y_pred)
    if (np.mean(y_true)<eps) and (np.mean(y_pred)<eps):
        return 1
    return (2*intersection) / (np.sum(y_true) + np.sum(y_pred))

In [ ]:
def resize_and_crop_center(im, sz=128):
    new_shape = (np.array(im.shape)/np.min(im.shape)*sz).astype(int)
    im = im.resize((new_shape[1], new_shape[0]))
    h, w = im.shape
    left = (w - sz)/2
    top = (h - sz)/2
    right = (w + sz)/2
    bottom = (h + sz)/2
    # Crop the center of the image
    im = im.crop((left, top, right, bottom))
    return im

In [ ]:
test_img_path = Path.cwd().parent/'data'/'MAGNETIC_TILE_SURFACE_DEFECTS'/'test_images'
test_mask_path = Path.cwd().parent/'data'/'MAGNETIC_TILE_SURFACE_DEFECTS'/'test_masks'

test_images = get_image_files(test_img_path)

dice_list = []
for img_path in tqdm(test_images):
    mask_path = test_mask_path/f'{img_path.stem}.png'
    im = Image.open(img_path)
    mask_im = Image.open(mask_path)
    im_resized = resize_and_crop_center(im, sz=128)
    y_true = resize_and_crop_center(mask_im, sz=128)
    y_true = np.array(y_true)
    y_pred, *_ = learn.predict(PILImage(im_resized))
    y_pred = np.array(y_pred)
    dice = dice_coef(y_true, y_pred)
    dice_list.append(dice)

dice_arr = np.array(dice_list)
dice_arr.mean()
